In [220]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_curve, auc
from keras.models import Sequential
from keras.layers import Dense,Activation,BatchNormalization,Dropout
from keras.callbacks import ModelCheckpoint,EarlyStopping,Callback
import tensorflow as tf
from sklearn.utils import class_weight
from sklearn.preprocessing import MinMaxScaler
from keras.optimizers import SGD
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
from sklearn.metrics import roc_curve,auc,log_loss, roc_auc_score
from bayes_opt import BayesianOptimization
from sklearn.decomposition import PCA

Using TensorFlow backend.


In [195]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [196]:
train.head()

,id,target,0,1,2,3,4,5,6,7,...,290,291,292,293,294,295,296,297,298,299
0,0,1.0,-0.098,2.165,0.681,-0.614,1.309,-0.455,-0.236,0.276,...,0.867,1.347,0.504,-0.649,0.672,-2.097,1.051,-0.414,1.038,-1.065
1,1,0.0,1.081,-0.973,-0.383,0.326,-0.428,0.317,1.172,0.352,...,-0.165,-1.695,-1.257,1.359,-0.808,-1.624,-0.458,-1.099,-0.936,0.973
2,2,1.0,-0.523,-0.089,-0.348,0.148,-0.022,0.404,-0.023,-0.172,...,0.013,0.263,-1.222,0.726,1.444,-1.165,-1.544,0.004,0.800,-1.211
3,3,1.0,0.067,-0.021,0.392,-1.637,-0.446,-0.725,-1.035,0.834,...,-0.404,0.640,-0.595,-0.966,0.900,0.467,-0.562,-0.254,-0.533,0.238
4,4,1.0,2.347,-0.831,0.511,-0.021,1.225,1.594,0.585,1.509,...,0.898,0.134,2.415,-0.996,-1.006,1.378,1.246,1.478,0.428,0.253


In [197]:
train.shape 


(250, 302)

In [198]:
test.shape

(19750, 301)

In [199]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Columns: 302 entries, id to 299
dtypes: float64(301), int64(1)
memory usage: 589.9 KB


In [200]:
train.describe()

,id,target,0,1,2,3,4,5,6,7,...,290,291,292,293,294,295,296,297,298,299
count,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,...,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000
mean,124.500000,0.640000,0.023292,-0.026872,0.167404,0.001904,0.001588,-0.007304,0.032052,0.078412,...,0.044652,0.126344,0.018436,-0.012092,-0.065720,-0.106112,0.046472,0.006452,0.009372,-0.128952
std,72.312977,0.480963,0.998354,1.009314,1.021709,1.011751,1.035411,0.955700,1.006657,0.939731,...,1.011416,0.972567,0.954229,0.960630,1.057414,1.038389,0.967661,0.998984,1.008099,0.971219
min,0.000000,0.000000,-2.319000,-2.931000,-2.477000,-2.359000,-2.566000,-2.845000,-2.976000,-3.444000,...,-2.804000,-2.443000,-2.757000,-2.466000,-3.287000,-3.072000,-2.634000,-2.776000,-3.211000,-3.500000
25%,62.250000,0.000000,-0.644750,-0.739750,-0.425250,-0.686500,-0.659000,-0.643750,-0.675000,-0.550750,...,-0.617000,-0.510500,-0.535750,-0.657000,-0.818500,-0.821000,-0.605500,-0.751250,-0.550000,-0.754250
50%,124.500000,1.000000,-0.015500,0.057000,0.184000,-0.016500,-0.023000,0.037500,0.060500,0.183500,...,0.067500,0.091000,0.057500,-0.021000,-0.009000,-0.079500,0.009500,0.005500,-0.009000,-0.132500
75%,186.750000,1.000000,0.677000,0.620750,0.805000,0.720000,0.735000,0.660500,0.783250,0.766250,...,0.797250,0.804250,0.631500,0.650250,0.739500,0.493000,0.683000,0.794250,0.654250,0.503250
max,249.000000,1.000000,2.567000,2.419000,3.392000,2.771000,2.901000,2.793000,2.546000,2.846000,...,2.865000,2.801000,2.736000,2.596000,2.226000,3.131000,3.236000,2.626000,3.530000,2.771000


In [201]:
train['target'].value_counts()

1.0    160
0.0     90
Name: target, dtype: int64

In [202]:
X=train.drop(['target','id'],axis=1,inplace=False)

In [203]:
X_test=test.drop('id',axis=1,inplace=False)

In [204]:
y=train['target']

In [63]:
pca = PCA(n_components=150)
train_PCA=pca.fit_transform(X)

In [64]:
np.sum(pca.explained_variance_ratio_)

0.9252929239783927

#### RF

In [ ]:
class_weight={}

In [264]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Since there was no preprocessing, we didn't need a pipeline here. Used anyway as best practice
modeling_pipeline = make_pipeline(RandomForestClassifier(n_estimators =10000,max_depth=4,max_features ='sqrt',n_jobs=-1))
cv_scores = cross_val_score(modeling_pipeline, X, y, scoring='roc_auc',cv=20)
print("Cross-val auc : %f" %cv_scores.mean())
print("Cross-val std : %f" %cv_scores.std())
print("Cross-val :",cv_scores)

Cross-val auc : 0.752188
Cross-val std : 0.147509
Cross-val : [0.975   0.775   0.7     0.375   0.7     0.85    0.875   0.725   0.85
 0.5     0.875   0.8125  0.875   0.78125 0.9375  0.78125 0.75    0.71875
 0.6875  0.5    ]


In [277]:
#sclf.fit( X, y)
y_pred=model.predict_proba(X_test)

In [278]:
y_pred

array([[0.31459898, 0.685401  ],
       [0.50341827, 0.49658173],
       [0.342085  , 0.657915  ],
       [0.45590293, 0.54409707],
       [0.2564901 , 0.7435099 ],
       [0.5833971 , 0.4166029 ],
       [0.40227854, 0.59772146],
       [0.25768524, 0.74231476],
       [0.39460427, 0.60539573],
       [0.3527364 , 0.6472636 ],
       [0.30412334, 0.69587666],
       [0.2232039 , 0.7767961 ],
       [0.35820472, 0.6417953 ],
       [0.56328833, 0.43671164],
       [0.28832376, 0.71167624],
       [0.2510332 , 0.7489668 ],
       [0.5452666 , 0.45473334],
       [0.1826312 , 0.8173688 ],
       [0.4491902 , 0.5508098 ],
       [0.4995103 , 0.5004897 ],
       [0.30904627, 0.69095373],
       [0.2944497 , 0.7055503 ],
       [0.39540523, 0.60459477],
       [0.4807049 , 0.5192951 ],
       [0.23096001, 0.76904   ],
       [0.40330124, 0.59669876],
       [0.22220558, 0.7777944 ],
       [0.6102414 , 0.38975856],
       [0.25533473, 0.74466527],
       [0.51740867, 0.48259133],
       [0.

In [279]:
test['target'] =  np.amax(y_pred,axis=1)

ValueError: Length of values does not match length of index

In [270]:
test.loc[:,['id','target']].to_csv('submissionRF.csv', index=False)

In [189]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import StackingClassifier

In [255]:
clf1 = RandomForestClassifier(n_estimators =10000,max_depth=2,max_features ='auto',n_jobs=-1)
clf2= xgb.XGBClassifier(n_estimators=2000,max_depth=7)
lf3 =LogisticRegression(penalty="l1",solver="liblinear")
sclf = StackingClassifier(classifiers=[clf1, clf2],meta_classifier=clf3,use_probas=True)

print('3-fold cross validation:\n')


scores = model_selection.cross_val_score(sclf, X, y,cv=10,scoring='roc_auc')
print(scores)

3-fold cross validation:

[0.84722222 0.60416667 0.66666667 0.79861111 0.79166667 0.625
 0.90972222 0.65972222 0.8125     0.63194444]


In [256]:
np.mean(scores)

0.7347222222222223

In [280]:
X_train, X_test2, y_train, y_test2 = train_test_split( X, y, test_size=0.2, random_state=42)

In [281]:
model = xgb.XGBClassifier(silent=False, 
                      scale_pos_weight=1,
                      learning_rate=0.01,  
                      colsample_bytree = 0.4,
                      subsample = 0.8,
                      objective='binary:logistic', 
                      n_estimators=2000, 
                      reg_alpha = 0.3,
                      max_depth=4, 
                      gamma=10)

In [ ]:
eval_set = [(X_train, y_train), (X_test, y_test)]
eval_metric = ["auc","error"]
model.fit(X_train, y_train, eval_metric=eval_metric, eval_set=eval_set, verbose=True)

[21:20:22] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 18 pruned nodes, max_depth=1
[0]	validation_0-auc:0.601333	validation_0-error:0.335	validation_1-auc:0.480952	validation_1-error:0.38
[21:20:22] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 18 pruned nodes, max_depth=1
[1]	validation_0-auc:0.716267	validation_0-error:0.345	validation_1-auc:0.738095	validation_1-error:0.28
[21:20:22] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 24 pruned nodes, max_depth=1
[2]	validation_0-auc:0.74304	validation_0-error:0.345	validation_1-auc:0.764762	validation_1-error:0.28
[21:20:22] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 10 pruned nodes, max_depth=1
[3]	validation_0-auc:0.764853	validation_0-error:0.315	validation_1-auc:0.757143	v

[21:20:23] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 16 pruned nodes, max_depth=2
[33]	validation_0-auc:0.93312	validation_0-error:0.205	validation_1-auc:0.822857	validation_1-error:0.24
[21:20:23] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 12 pruned nodes, max_depth=3
[34]	validation_0-auc:0.940053	validation_0-error:0.205	validation_1-auc:0.819048	validation_1-error:0.24
[21:20:23] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 12 pruned nodes, max_depth=3
[35]	validation_0-auc:0.946773	validation_0-error:0.205	validation_1-auc:0.817143	validation_1-error:0.22
[21:20:23] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 16 pruned nodes, max_depth=1
[36]	validation_0-auc:0.947413	validation_0-error:0.205	validation_1-auc:0.8247

[21:20:23] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 18 pruned nodes, max_depth=0
[66]	validation_0-auc:0.95328	validation_0-error:0.21	validation_1-auc:0.819048	validation_1-error:0.2
[21:20:23] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 22 pruned nodes, max_depth=2
[67]	validation_0-auc:0.9552	validation_0-error:0.21	validation_1-auc:0.820952	validation_1-error:0.22
[21:20:23] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 18 pruned nodes, max_depth=1
[68]	validation_0-auc:0.9568	validation_0-error:0.21	validation_1-auc:0.820952	validation_1-error:0.22
[21:20:23] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 22 pruned nodes, max_depth=0
[69]	validation_0-auc:0.9568	validation_0-error:0.21	validation_1-auc:0.820952	validati

[21:20:23] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 22 pruned nodes, max_depth=0
[98]	validation_0-auc:0.962667	validation_0-error:0.225	validation_1-auc:0.834286	validation_1-error:0.2
[21:20:23] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 20 pruned nodes, max_depth=1
[99]	validation_0-auc:0.96192	validation_0-error:0.225	validation_1-auc:0.832381	validation_1-error:0.2
[21:20:23] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 16 pruned nodes, max_depth=1
[100]	validation_0-auc:0.96256	validation_0-error:0.225	validation_1-auc:0.834286	validation_1-error:0.18
[21:20:23] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 20 pruned nodes, max_depth=0
[101]	validation_0-auc:0.96256	validation_0-error:0.23	validation_1-auc:0.834286	

[21:20:23] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 12 pruned nodes, max_depth=3
[131]	validation_0-auc:0.96864	validation_0-error:0.22	validation_1-auc:0.84381	validation_1-error:0.18
[21:20:23] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 22 pruned nodes, max_depth=0
[132]	validation_0-auc:0.96864	validation_0-error:0.225	validation_1-auc:0.84381	validation_1-error:0.18
[21:20:23] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 16 pruned nodes, max_depth=0
[133]	validation_0-auc:0.96864	validation_0-error:0.225	validation_1-auc:0.84381	validation_1-error:0.18
[21:20:23] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 16 pruned nodes, max_depth=2
[134]	validation_0-auc:0.96768	validation_0-error:0.22	validation_1-auc:0.847619	v

...

[21:20:31] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 20 pruned nodes, max_depth=0
[1318]	validation_0-auc:0.990293	validation_0-error:0.155	validation_1-auc:0.832381	validation_1-error:0.16
[21:20:31] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 22 pruned nodes, max_depth=0
[1319]	validation_0-auc:0.990293	validation_0-error:0.155	validation_1-auc:0.832381	validation_1-error:0.16
[21:20:31] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 18 pruned nodes, max_depth=0
[1320]	validation_0-auc:0.990293	validation_0-error:0.155	validation_1-auc:0.832381	validation_1-error:0.16
[21:20:31] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 22 pruned nodes, max_depth=0
[1321]	validation_0-auc:0.990293	validation_0-error:0.155	validation_1-a

[21:20:31] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 22 pruned nodes, max_depth=0
[1350]	validation_0-auc:0.990293	validation_0-error:0.155	validation_1-auc:0.832381	validation_1-error:0.16
[21:20:31] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 18 pruned nodes, max_depth=2
[1351]	validation_0-auc:0.990827	validation_0-error:0.16	validation_1-auc:0.830476	validation_1-error:0.16
[21:20:31] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 22 pruned nodes, max_depth=0
[1352]	validation_0-auc:0.990827	validation_0-error:0.16	validation_1-auc:0.830476	validation_1-error:0.16
[21:20:31] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 20 pruned nodes, max_depth=0
[1353]	validation_0-auc:0.990827	validation_0-error:0.16	validation_1-auc:

[21:20:31] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 18 pruned nodes, max_depth=0
[1382]	validation_0-auc:0.990507	validation_0-error:0.16	validation_1-auc:0.828571	validation_1-error:0.16
[21:20:31] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 20 pruned nodes, max_depth=0
[1383]	validation_0-auc:0.990507	validation_0-error:0.16	validation_1-auc:0.828571	validation_1-error:0.16
[21:20:31] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 18 pruned nodes, max_depth=0
[1384]	validation_0-auc:0.990507	validation_0-error:0.16	validation_1-auc:0.828571	validation_1-error:0.16
[21:20:31] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 20 pruned nodes, max_depth=0
[1385]	validation_0-auc:0.990507	validation_0-error:0.16	validation_1-auc:0

[21:20:31] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 14 pruned nodes, max_depth=0
[1414]	validation_0-auc:0.990507	validation_0-error:0.15	validation_1-auc:0.828571	validation_1-error:0.16
[21:20:31] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 16 pruned nodes, max_depth=0
[1415]	validation_0-auc:0.990507	validation_0-error:0.15	validation_1-auc:0.828571	validation_1-error:0.16
[21:20:31] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 22 pruned nodes, max_depth=0
[1416]	validation_0-auc:0.990507	validation_0-error:0.15	validation_1-auc:0.828571	validation_1-error:0.16
[21:20:31] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 18 pruned nodes, max_depth=0
[1417]	validation_0-auc:0.990507	validation_0-error:0.15	validation_1-auc:0

[21:20:31] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 24 pruned nodes, max_depth=0
[1446]	validation_0-auc:0.990507	validation_0-error:0.15	validation_1-auc:0.828571	validation_1-error:0.16
[21:20:31] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 16 pruned nodes, max_depth=0
[1447]	validation_0-auc:0.990507	validation_0-error:0.15	validation_1-auc:0.828571	validation_1-error:0.16
[21:20:31] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 22 pruned nodes, max_depth=0
[1448]	validation_0-auc:0.990507	validation_0-error:0.15	validation_1-auc:0.828571	validation_1-error:0.16
[21:20:31] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 20 pruned nodes, max_depth=0
[1449]	validation_0-auc:0.990507	validation_0-error:0.15	validation_1-auc:0

In [276]:
model.fit(X_train, y_train, eval_metric=eval_metric, eval_set=eval_set, verbose=True)

[21:16:54] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 18 pruned nodes, max_depth=1
[0]	validation_0-auc:0.601333	validation_0-error:0.335	validation_1-auc:0.480952	validation_1-error:0.38
[21:16:54] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 18 pruned nodes, max_depth=1
[1]	validation_0-auc:0.716267	validation_0-error:0.345	validation_1-auc:0.738095	validation_1-error:0.28
[21:16:54] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 24 pruned nodes, max_depth=1
[2]	validation_0-auc:0.74304	validation_0-error:0.345	validation_1-auc:0.764762	validation_1-error:0.28
[21:16:54] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 10 pruned nodes, max_depth=1
[3]	validation_0-auc:0.764853	validation_0-error:0.315	validation_1-auc:0.757143	v

[21:16:54] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 16 pruned nodes, max_depth=2
[33]	validation_0-auc:0.93312	validation_0-error:0.205	validation_1-auc:0.822857	validation_1-error:0.24
[21:16:55] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 12 pruned nodes, max_depth=3
[34]	validation_0-auc:0.940053	validation_0-error:0.205	validation_1-auc:0.819048	validation_1-error:0.24
[21:16:55] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 12 pruned nodes, max_depth=3
[35]	validation_0-auc:0.946773	validation_0-error:0.205	validation_1-auc:0.817143	validation_1-error:0.22
[21:16:55] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 16 pruned nodes, max_depth=1
[36]	validation_0-auc:0.947413	validation_0-error:0.205	validation_1-auc:0.8247

[21:16:55] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 18 pruned nodes, max_depth=0
[66]	validation_0-auc:0.95328	validation_0-error:0.21	validation_1-auc:0.819048	validation_1-error:0.2
[21:16:55] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 22 pruned nodes, max_depth=2
[67]	validation_0-auc:0.9552	validation_0-error:0.21	validation_1-auc:0.820952	validation_1-error:0.22
[21:16:55] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 18 pruned nodes, max_depth=1
[68]	validation_0-auc:0.9568	validation_0-error:0.21	validation_1-auc:0.820952	validation_1-error:0.22
[21:16:55] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 22 pruned nodes, max_depth=0
[69]	validation_0-auc:0.9568	validation_0-error:0.21	validation_1-auc:0.820952	validati

[21:16:55] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 20 pruned nodes, max_depth=1
[99]	validation_0-auc:0.96192	validation_0-error:0.225	validation_1-auc:0.832381	validation_1-error:0.2
[21:16:55] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 16 pruned nodes, max_depth=1
[100]	validation_0-auc:0.96256	validation_0-error:0.225	validation_1-auc:0.834286	validation_1-error:0.18
[21:16:55] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 20 pruned nodes, max_depth=0
[101]	validation_0-auc:0.96256	validation_0-error:0.23	validation_1-auc:0.834286	validation_1-error:0.18
[21:16:55] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 16 pruned nodes, max_depth=1
[102]	validation_0-auc:0.960533	validation_0-error:0.225	validation_1-auc:0.83809

[131]	validation_0-auc:0.96864	validation_0-error:0.22	validation_1-auc:0.84381	validation_1-error:0.18
[21:16:55] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 22 pruned nodes, max_depth=0
[132]	validation_0-auc:0.96864	validation_0-error:0.225	validation_1-auc:0.84381	validation_1-error:0.18
[21:16:55] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 16 pruned nodes, max_depth=0
[133]	validation_0-auc:0.96864	validation_0-error:0.225	validation_1-auc:0.84381	validation_1-error:0.18
[21:16:55] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 16 pruned nodes, max_depth=2
[134]	validation_0-auc:0.96768	validation_0-error:0.22	validation_1-auc:0.847619	validation_1-error:0.18
[21:16:55] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 16 pru

...

[21:17:05] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 16 pruned nodes, max_depth=0
[1896]	validation_0-auc:0.991787	validation_0-error:0.13	validation_1-auc:0.832381	validation_1-error:0.18
[21:17:05] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 22 pruned nodes, max_depth=0
[1897]	validation_0-auc:0.991787	validation_0-error:0.13	validation_1-auc:0.832381	validation_1-error:0.18
[21:17:05] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 22 pruned nodes, max_depth=0
[1898]	validation_0-auc:0.991787	validation_0-error:0.13	validation_1-auc:0.832381	validation_1-error:0.18
[21:17:05] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 22 pruned nodes, max_depth=0
[1899]	validation_0-auc:0.991787	validation_0-error:0.13	validation_1-auc:0

[21:17:05] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 14 pruned nodes, max_depth=0
[1928]	validation_0-auc:0.992107	validation_0-error:0.125	validation_1-auc:0.830476	validation_1-error:0.2
[21:17:05] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 26 pruned nodes, max_depth=0
[1929]	validation_0-auc:0.992107	validation_0-error:0.125	validation_1-auc:0.830476	validation_1-error:0.2
[21:17:05] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 20 pruned nodes, max_depth=0
[1930]	validation_0-auc:0.992107	validation_0-error:0.125	validation_1-auc:0.830476	validation_1-error:0.2
[21:17:05] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 24 pruned nodes, max_depth=0
[1931]	validation_0-auc:0.992107	validation_0-error:0.125	validation_1-auc:

[21:17:05] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 20 pruned nodes, max_depth=0
[1960]	validation_0-auc:0.992107	validation_0-error:0.125	validation_1-auc:0.830476	validation_1-error:0.2
[21:17:05] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 22 pruned nodes, max_depth=0
[1961]	validation_0-auc:0.992107	validation_0-error:0.125	validation_1-auc:0.830476	validation_1-error:0.2
[21:17:05] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 22 pruned nodes, max_depth=0
[1962]	validation_0-auc:0.992107	validation_0-error:0.125	validation_1-auc:0.830476	validation_1-error:0.18
[21:17:05] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 22 pruned nodes, max_depth=0
[1963]	validation_0-auc:0.992107	validation_0-error:0.125	validation_1-auc

[21:17:06] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 16 pruned nodes, max_depth=0
[1992]	validation_0-auc:0.992107	validation_0-error:0.125	validation_1-auc:0.830476	validation_1-error:0.18
[21:17:06] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 20 pruned nodes, max_depth=0
[1993]	validation_0-auc:0.992107	validation_0-error:0.125	validation_1-auc:0.830476	validation_1-error:0.18
[21:17:06] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 18 pruned nodes, max_depth=0
[1994]	validation_0-auc:0.992107	validation_0-error:0.125	validation_1-auc:0.830476	validation_1-error:0.18
[21:17:06] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 22 pruned nodes, max_depth=0
[1995]	validation_0-auc:0.992107	validation_0-error:0.125	validation_1-a

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.4, gamma=10, learning_rate=0.01,
       max_delta_step=0, max_depth=4, min_child_weight=1, missing=None,
       n_estimators=2000, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0.3,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=False,
       subsample=0.8)